# Retrieval Augmented Generation with Amazon Bedrock - Enhancing Chat Applications with RAG

> *PLEASE NOTE: This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

---

## Chat with LLMs Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience for your customers. Chatbots can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users.

The key technical detail which we need to include in our system to enable a chat feature is conversational memory. This way, customers can ask follow up questions and the LLM will understand what the customer has already said in the past. The image below shows how this is orchestrated at a high level.

![Amazon Bedrock - Conversational Interface](./images/chatbot_bedrock.png)

## Extending Chat with RAG

However, in our workshop's situation, we want to be able to enable a customer to ask follow up questions regarding documentation we provide through RAG. This means we need to build a system which has conversational memory AND contextual retrieval built into the text generation.

![4](./images/context-aware-chatbot.png)

Let's get started!

---

## Setup `boto3` Connection

#### Libraries needed for the installs

In [1]:
%pip install  \
    "langchain>=0.1.11" \
    "transformers>=4.24,<5" \
    "faiss-cpu>=1.7.4,<2" \
    "pypdf>=3.8,<4" \
    pinecone-client==2.2.4 \
    apache-beam==2.52. \
    tiktoken==0.5.2 \
    "ipywidgets>=7,<8" \
    matplotlib==3.8.2 \
    anthropic==0.9.0 \
    llama-index==0.9.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import os
from IPython.display import Markdown, display

region = os.environ.get("AWS_REGION")
boto3_bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=region,
)

### Use the following COT prompt to test 

In [3]:
prompt = "Human: You are a supply chain inspector. Your job is to assess the risk of a supplier. You measure the supplier risk by evaluating the supplier across three dimensions: country, size, and reputation.\nCountry - north America and west Europe countries are considered low risk. The rest of the world is considered medium risk.\nSize - supplier with over 1000 employees is low risk. Supplier with 50 to 999 employees is medium risk, and a supplier with under 50 employees is high risk.\nReputation - reputation scores are between 1 to 10 where a score of 1 to 3 is low risk, a score of 4 to 7 is medium risk, and a reputation score of 8 to 10 is high risk.\nThe risk formula is to take the maximum risk across the three dimensions.\n##\nExample:\nSupplier: A\nCountry: Chad\nSize: 30\nReputation: 8\nLet's think step by step:\nChad is not in North America or West Europe therefore country risk is medium.\nA size of 30 is below 50 and therefore considered high risk.\nA reputation score of 8 is between 8 to 10 and therefore considered high risk.\nFinal Answer taking the maximum risk among all: Supplier A is at High risk.\n##\nSupplier: B\nCountry: USA\nSize: 40\nReputation: 2\nLet's think step by step: \nAssistant: "


model_output = " Okay, let's evaluate Supplier B:\n\n- Country: USA is in North America, so the country risk is low.\n\n- Size: 40 employees is below 50, so the size risk is high. \n\n- Reputation: A score of 2 is between 1-3, so the reputation risk is low.\n\nTo determine the overall risk, we take the maximum risk across the three dimensions. \n\nThe maximum risk for Supplier B is high due to its small size.\n\nTherefore, my assessment is that Supplier B is at high risk overall."

---
## Using LangChain for Conversation Memory

We will use LangChain's `ConversationBufferMemory` class provides an easy way to capture conversational memory for LLM chat applications. Let's check out an example of Claude being able to retrieve context through conversational memory below.

Similar to the last workshop, we will use both a prompt template and a LangChain LLM for this example. Note that this time our prompt template includes a `{history}` variable where our chat history will be included to the prompt.

In [4]:
from langchain import PromptTemplate

CHAT_PROMPT_TEMPLATE = '''You are a helpful conversational assistant.
{history}

Human: {human_input}

Assistant:
'''
PROMPT = PromptTemplate.from_template(CHAT_PROMPT_TEMPLATE)

In [5]:
from langchain.llms import Bedrock

llm = Bedrock(
    client=boto3_bedrock,
    model_id="anthropic.claude-instant-v1",
    model_kwargs={
        "max_tokens_to_sample": 500,
        "temperature": 0.9,
    },
)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `Bedrock` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import BedrockLLM`.
  warn_deprecated(


The `ConversationBufferMemory` class is instantiated here and you will notice that we use Claude specific human and assistant prefixes. When we initialize the memory, the history is blank.

In [6]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(human_prefix="\nHuman", ai_prefix="\nAssistant")
history = memory.load_memory_variables({})['history']
print(history)

We now ask Claude a simple question "How can I check for imbalances in my model?". The LLM responds to the question and we can use the `add_user_message` and `add_ai_message` functions to save the input and output into memory. We can then retrieve the entire conversation history and print the response. Currently the model will still return answer using the data it was trained upon. Further will examine how to get a curated answer using our own FAq's

In [7]:
human_input = 'How can I check for imbalances in my model?'

prompt_data = PROMPT.format(human_input=human_input, history=history)
ai_output = llm(prompt_data)

memory.chat_memory.add_user_message(human_input)
memory.chat_memory.add_ai_message(ai_output.strip())
history = memory.load_memory_variables({})['history']
display(Markdown(f'{history}'))

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



Human: How can I check for imbalances in my model?

Assistant: Here are some things you can do to check for imbalances in your model:

- Check the data used to train the model for biases or imbalances. Look at attributes like gender, race, age, location, etc. and ensure all groups are adequately represented. Imbalanced data can lead to models that don't generalize well.

- Evaluate model performance on different demographic groups separately. Check that accuracy, error rates, etc. are similar across groups. Large differences could indicate unfairness. 

- Analyze model explanations/interpretations for potential biases. Look at what factors the model uses to make predictions and see if certain attributes have outsized influence in a harmful way.

- Conduct fairness testing with tools like model cards, bias profiles, and fairness metrics. This helps systematically check for unfair treatment of individuals or groups.

- Get feedback from diverse stakeholders on any potential harms or biases they see. Real users from varied backgrounds can help identify issues the model builders may have missed.  

- Consider techniques like re-weighting the training process, regularization, or post-processing to reduce the influence of sensitive attributes on predictions. This can help address some imbalances.

- Benchmark against other models, especially those designed specifically for fairness, to compare treatment of different subgroups.

Regular monitoring and auditing of models can help catch issues with imbalances or unfairness over time as usage expands. Being diligent about inclusiveness in data and model performance is important.

Now we will ask a follow up question about the kind of imbalances does it detect and save the input and outputs again. Notice how the model is able to understand that when the human says "it", because it has access to the context of the chat history, the model is able to accurately understand what the user is asking about.

In [8]:
human_input = 'What kind does it detect?'

prompt_data = PROMPT.format(human_input=human_input, history=history)
ai_output = llm(prompt_data)

memory.chat_memory.add_user_message(human_input)
memory.chat_memory.add_ai_message(ai_output.strip())
#display(Markdown(f'{history}'))
display(Markdown(f'{ai_output}'))

 There are a few main types of imbalances that can be detected when checking a model:

- Demographic imbalances - This refers to discrepancies in how well the model performs across different demographic groups like gender, race, age, location etc. Metrics here look at variable performance across subgroups.

- Class imbalance - When one or more classes/labels are under-represented in the training data compared to others. This can skew the model to focus more on majority classes. 

- Attribute imbalance - Certain input attributes may have a disproportionately large influence on predictions in a way that harms some groups. Detecting dominant attributes is important.

- Treatment imbalance - Related to demographic imbalances, but focuses on differences in how individuals from various subgroups are treated by the model. For example, higher error rates or unfair outcomes.

- Selection bias - The data used to train the model may not properly represent the target population the model is intended to serve, neglecting certain subgroups. 

- Measurement bias - Errors or inconsistencies in how attributes are measured or defined could skew the modeling process unfairly against some individuals or demographics.

So in summary, checks focus on variable performance, class/attribute distributions, influential factors, selection issues and potential unfair treatment when evaluating models for different types of imbalances. The goal is to ensure equitable, unbiased outcomes.

---
## Creating a class to help facilitate conversation

To help create some structure around these conversations, we create a custom `Conversation` class below. This class will hold a stateful conversational memory and be the base for conversational RAG later.

In [9]:
class Conversation:
    def __init__(self, client, model_id: str="anthropic.claude-instant-v1") -> None:
        """instantiates a new rag based conversation

        Args:
            model_id (str, optional): which bedrock model to use for the conversational agent. Defaults to "anthropic.claude-instant-v1".
        """

        # instantiate memory
        self.memory = ConversationBufferMemory(human_prefix="\nHuman", ai_prefix="\nAssistant")

        # instantiate LLM connection
        self.llm = Bedrock(
            client=client,
            model_id=model_id,
            model_kwargs={
                "max_tokens_to_sample": 500,
                "temperature": 0.9,
            },
        )

    def ai_respond(self, user_input: str=None):
        """responds to the user input in the conversation with context used

        Args:
            user_input (str, optional): user input. Defaults to None.

        Returns:
            ai_output (str): response from AI chatbot
        """

        # format the prompt with chat history and user input
        history = self.memory.load_memory_variables({})['history']
        llm_input = PROMPT.format(history=history, human_input=user_input)

        # respond to the user with the LLM
        ai_output = self.llm(llm_input).strip()

        # store the input and output
        self.memory.chat_memory.add_user_message(user_input)
        self.memory.chat_memory.add_ai_message(ai_output.strip())

        return ai_output

Let's see the class in action with two contextual questions. Again, notice the model is able to correctly interpret the context because it has memory of the conversation.

In [10]:
chat = Conversation(client=boto3_bedrock)

In [11]:
output = chat.ai_respond('How can I check for imbalances in my model?')
display(Markdown(f'{output}'))

Here are a few things you can do to check for imbalances in your machine learning model:

- Compute demographic parity metrics like statistical parity or equalized odds. These quantify how well a model treats different demographic groups (e.g. gender, race) equally in terms of predicted outcomes. Imbalances will show up as differences across groups.

- Look at precision, recall and F1 scores stratified by demographic groups. Imbalances may show up as differences in these evaluation metrics across groups. For example, higher recall but lower precision for one group.

- Visualize the model's predictions on a sample of test data and check if it behaves similarly across demographic attributes. Look for inconsistencies or bias against certain groups.

- audit model explanations/attentions to see if they rely on protected attributes like gender or race when making predictions. Reliance is a sign of potential unfair bias.

- collect bias metrics like disparate impact which quantify differences in false positive/negative rates between groups. Imbalance is indicated by disparities. 

- reweight or resample the training data to create demographic parity, then retrain and check if overall accuracy decreases significantly. Large drops suggest overdependence on imbalanced attributes.

The goal is to systematically analyze model performance, explanations and outcomes across demographic slices to detect any harmful biases or imbalances during development. Continual monitoring after deployment is also important.

In [12]:
output = chat.ai_respond('What kind does it detect?')
display(Markdown(f'{output}'))

Here are some common types of imbalances that can be detected when checking a machine learning model:

- Statistical imbalances - When the distribution of labels/classes is uneven across different demographic groups in the training data. This can result in poorer performance for under-represented groups.

- Precision/recall imbalances - If a model has significantly higher precision but lower recall (or vice versa) for certain groups. Indicates some groups may be over or under-predicted.  

- Threshold imbalances - Differences in decision thresholds applied by the model across groups can cause disparities. For example, higher thresholds for one group.

- Feature reliance imbalances - If model explanations or feature importances reveal stronger reliance on certain features correlated with protected attributes like gender or race. 

- Disparate treatment - When individuals from different demographic groups with similar characteristics receive different treatment or predictions from the model.

- Disparate impact - When a model predicts equal treatment but results in unequal adverse impact or harm between demographic groups. For example, higher error rates for a particular group.

- Masking and redirection biases - When a model learns to mask or redirect predictions based on explicit/implicit cues about an individual's demographic group rather than their qualifications.

Regular auditing helps uncover statistical, predictive performance, decision threshold, reliance, treatment and impact imbalances between groups. This ensures fair and representative modeling.

---
## Combining RAG with Conversation

Now that we have a conversational system built, lets incorporate the RAG system we built in notebook 02 into the chat paradigm. 

First, we will create the same vector store with LangChain and FAISS from the last notebook.

Our goal is to create a curated response from the model and only use the FAQ's we have provided.

In [13]:
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS


# create instantiation to embedding model
embedding_model = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id="amazon.titan-embed-text-v1"
)

# create vector store
vs = FAISS.load_local('./faiss-index/langchain', embedding_model, allow_dangerous_deserialization=True)

RuntimeError: Error in faiss::FileIOReader::FileIOReader(const char*) at /project/faiss/faiss/impl/io.cpp:67: Error: 'f' failed: could not open faiss-index/langchain/index.faiss for reading: No such file or directory

### Visualize Semantic Search 

⚠️ ⚠️ ⚠️ This section is for Advanced Practioners. Please feel free to run through these cells and come back later to re-examine the concepts ⚠️ ⚠️ ⚠️ 

Let's see how the semantic search works:
1. First we calculate the embeddings vector for the query, and
2. then we use this vector to do a similarity search on the store


##### Citation
We will also be able to get the `citation` or the underlying documents which our Vector Store matched to our query. This is useful for debugging and also measuring the quality of the vector stores. let us look at how the underlying Vector store calculates the matches

##### Vector DB Indexes
One of the key components of the Vector DB is to be able to retrieve documents matching the query with accuracy and speed. There are multiple algorithims for the same and some examples can be [read here](https://thedataquarry.com/posts/vector-db-3/) 

In [ ]:
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')
#- helpful function to display in tabular format

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td>%s</td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

In [ ]:

v = embedding_model.embed_query("How can I check for imbalances in my model?")
print(v[0:10])
results = vs.similarity_search_by_vector(v, k=2)
display(Markdown('Let us look at the documents which had the relevant information pertaining to our query'))
for r in results:
    display(Markdown(f'{r.page_content}'))
    display(Markdown(f'------------------------------------'))

#### Similarity Search

##### Distance scoring in Vector Data bases
[Distance scores](https://weaviate.io/blog/distance-metrics-in-vector-search) are the key in vector searches. Here are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance ( Squared Euclidean) . Therefore, a lower score is better. Further in FAISS we have similarity_search_with_score (ranked by distance: low to high) and similarity_search_with_relevance_scores ( ranked by relevance: high to low) with both using the distance strategy. The similarity_search_with_relevance_scores calculates the relevance score as 1 - score. For more details of the various distance scores [read here](https://milvus.io/docs/metric.md)


In [ ]:
display(Markdown(f"##### Let us look at the documents based on {vs.distance_strategy.name} which will be used to answer our question 'What kind of bias does Clarify detect ?'"))

context = vs.similarity_search('What kind of bias does Clarify detect ?', k=2)
#-  langchain.schema.document.Document
display(Markdown(f'------------------------------------'))
list_context = [[doc.page_content, doc.metadata] for doc in context]
list_context.insert(0, ['Documents', 'Meta-data'])
display_table(list_context)

Let us first look at the Page context and the meta data associated with the documents. Now let us look at the L2 scores based on the distance scoring as explained above. Lower score is better

In [ ]:
#- relevancy of the documents
results = vs.similarity_search_with_score("What kind of bias does Clarify detect ?", k=2, fetch_k=3)
display(Markdown(f'##### Similarity Search Table with relevancy score.'))
display(Markdown(f'------------------------------------'))   
results.insert(0,['Documents', 'Relevancy Score'])
display_table(results)

#### Marginal Relevancy score

Maximal Marginal Relevance  has been introduced in the paper [The Use of MMR, Diversity-Based Reranking for Reordering Documents and Producing Summaries](https://www.cs.cmu.edu/~jgc/publication/The_Use_MMR_Diversity_Based_LTMIR_1998.pdf). Maximal Marginal Relevance tries to reduce the redundancy of results while at the same time maintaining query relevance of results for already ranked documents/phrases etc. In the below results since we have a very limited data set it might not make a difference but for larger data sets the query will theoritically run faster while still preserving the over all relevancy of the documents

In [ ]:
#- normalizing the relevancy
display(Markdown('##### Let us look at MRR scores'))
results = vs.max_marginal_relevance_search_with_score_by_vector(embedding_model.embed_query("What kind of bias does Clarify detect ?"), k=3)
results.insert(0, ["Document", "MRR Score"])
display_table(results)
  

#### Update embeddings of the Vector Databases

Update of documents happens all the time and we have multiple versions of the documents. Which means we need to also factor how do we update the embeddings in our Vector Data bases. Fortunately we have and can leverage the meta data to update embeddings

The key steps are:
1. Load the new embeddings and add the meta data stating the version as 2
2. Merge to the exisiting Vector database
3. Run the query using the filter to only search in the new index and get the latest documents for the same query


In [ ]:
# create vector store
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import Document
loader = CSVLoader(
    file_path="./data/sagemaker/sm_faq_v2.csv",
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        "fieldnames": ["Question", "Answer"],
    },
)

#docs_split = loader.load()
docs_split = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator=",").split_documents(loader.load())
list_of_documents = [Document(page_content=doc.page_content, metadata=dict(page='v2')) for idx, doc in enumerate(docs_split)]
print(f"Number of split docs={len(docs_split)}")
db = FAISS.from_documents(list_of_documents, embedding_model)

#### Run a query against version 2 of the documents
Let us run the query agsint our exisiting vector data base and we will see the the exisiting or the version 1 of the documents coming back. If we run with the filter since those do not exist in our vector Database we will see no results returned or an empty list back


In [ ]:
# Run the query with requesting data from version 2 which does not exist
vs = FAISS.load_local('./faiss-index/langchain/', embedding_model, allow_dangerous_deserialization=True)
search_query = "How can I check for imbalances in my model?"
#print(f"Running with v1 of the documents we get response of {vs.similarity_search_with_score(query=search_query, k=1, fetch_k=4)}")
print("------\n")
print(f"Running the query with V2 of the document we get {vs.similarity_search_with_score(query=search_query, filter=dict(page='v2'), k=1)}:")


#### Add a new version of the document
We will create the version 2 of the documents and use meta data to add to our original index. Once done we will then apply a filter in our query which will return to us the documents newly added. Run the query now after adding version of the documents

We will also examine a way to speed up our searches and queries and look at another way to narrow the search using the  fetch_k parameter when calling similarity_search with filters. Usually you would want the fetch_k to be more than the k parameter. This is because the fetch_k parameter is the number of documents that will be fetched before filtering. If you set fetch_k to a low number, you might not get enough documents to filter from.

In [ ]:
# - now let us add version 2 of the data set and run query from that

vs.merge_from(db)

#### Query complete merged data base with no filters
Run the query against the fully merged DB without any filters for the meta data and we see that it returns the top results of the new V2 data and also the top results of the v1 data. Essentially it will match and return data closest to the query

In [ ]:
# - run the query again
search_query_v2 = "How can I check for imbalances in my model?"
results_with_scores = vs.similarity_search_with_score(search_query_v2, k=2, fetch_k=3)
results_with_scores = [[doc.page_content, doc.metadata, score] for doc, score in results_with_scores]
results_with_scores.insert(0, ['Document', 'Meta-Data', 'Score'])
display_table(results_with_scores)

#### Query with Filter
Now we will ask to search only against the version 2 of the data and use filter criteria against it

In [ ]:
# - run the query again
search_query_v2 = "How can I check for imbalances in my model?"
results_with_scores = vs.similarity_search_with_score(search_query_v2, filter=dict(page='v2'), k=2, fetch_k=3)
results_with_scores = [[doc.page_content, doc.metadata, score] for doc, score in results_with_scores]
results_with_scores.insert(0, ['Document', 'Meta-Data', 'Score'])
display_table(results_with_scores)

#### Query for new data
Now let us ask a question which exists only on the version 2 of the document

In [ ]:
# - now let us ask a question which ONLY exits in the version 2 of the document
search_query_v2 = "Can i use Quantum computing?"
results_with_scores = vs.similarity_search_with_score(query=search_query_v2, filter=dict(page='v2'), k=1, fetch_k=3)
results_with_scores = [[doc.page_content, doc.metadata, score] for doc, score in results_with_scores]
results_with_scores.insert(0, ['Document', 'Meta-Data', 'Score'])
display_table(results_with_scores)

### Let us continue to build our chatbot

The prompt template is now altered to include both conversation memory as well as chat history as inputs along with the human input. Notice how the prompt also instructs Claude to not answer questions which it does not have the context for. This helps reduce hallucinations which is extremely important when creating end user facing applications which need to be factual.

In [ ]:
# re-create vector store and continue
vs = FAISS.load_local('./faiss-index/langchain/', embedding_model, allow_dangerous_deserialization=True)

In [ ]:
RAG_TEMPLATE = """You are a helpful conversational assistant.

If you are unsure about the answer OR the answer does not exist in the context, respond with
"Sorry but I do not understand your request. I am still learning so I appreciate your patience! 😊
NEVER make up the answer.

If the human greets you, simply introduce yourself.

The context will be placed in <context></context> XML tags. 

<context>{context}</context>

Do not include any xml tags in your response.

{history}

Human: {input}

Assistant:
"""
PROMPT = PromptTemplate.from_template(RAG_TEMPLATE)

The new `ConversationWithRetrieval` class now includes a `get_context` function which searches our vector database based on the human input and combines it into the base prompt.

In [ ]:
class ConversationWithRetrieval:
    def __init__(self, client, vector_store: FAISS=None, model_id: str="anthropic.claude-instant-v1") -> None:
        """instantiates a new rag based conversation

        Args:
            vector_store (FAISS, optional): pre-populated vector store for searching context. Defaults to None.
            model_id (str, optional): which bedrock model to use for the conversational agent. Defaults to "anthropic.claude-instant-v1".
        """

        # store vector store
        self.vector_store = vector_store
        
        # instantiate memory
        self.memory = ConversationBufferMemory(human_prefix="Human", ai_prefix="Assistant")

        # instantiate LLM connection
        self.llm = Bedrock(
            client=client,
            model_id=model_id,
            model_kwargs={
                "max_tokens_to_sample": 500,
                "temperature": 0.0,
            },
        )

    def ai_respond(self, user_input: str=None):
        """responds to the user input in the conversation with context used

        Args:
            user_input (str, optional): user input. Defaults to None.

        Returns:
            ai_output (str): response from AI chatbot
            search_results (list): context used in the completion
        """

        # format the prompt with chat history and user input
        context_string, search_results = self.get_context(user_input)
        history = self.memory.load_memory_variables({})['history']
        llm_input = PROMPT.format(history=history, input=user_input, context=context_string)

        # respond to the user with the LLM
        ai_output = self.llm(llm_input).strip()

        # store the input and output
        self.memory.chat_memory.add_user_message(user_input)
        self.memory.chat_memory.add_ai_message(ai_output.strip())

        return ai_output, search_results

    def get_context(self, user_input, k=5):
        """returns context used in the completion

        Args:
            user_input (str): user input as a string
            k (int, optional): number of results to return. Defaults to 5.

        Returns:
            context_string (str): context used in the completion as a string
            search_results (list): context used in the completion as a list of Document objects
        """
        search_results = self.vector_store.similarity_search(
            user_input, k=k
        )
        context_string = '\n\n'.join([f'Document {ind+1}: ' + i.page_content for ind, i in enumerate(search_results)])
        return context_string, search_results

Now the model can answer some specific domain questions based on our document database!

In [ ]:
chat = ConversationWithRetrieval(boto3_bedrock, vs)

In [ ]:
output, context = chat.ai_respond('How can I check for imbalances in my model?')
display(Markdown(f'{output}'))

In [ ]:
output, context = chat.ai_respond('What kind does it detect?')
display(Markdown(f'** Ai Assistant Answer: ** \n{output}'))
display(Markdown(f'\n\n** Relevant Documentation: ** \n{context}'))

--- 
## Using LangChain for Orchestration of RAG

Beyond the primitive classes for prompt handling and conversational memory management, LangChain also provides a framework for [orchestrating RAG flows](https://python.langchain.com/docs/expression_language/cookbook/retrieval) with what purpose built "chains". In this section, we will see how to be a retrieval chain with LangChain which is more comprehensive and robust than the original retrieval system we built above.

The workflow we used above follows the following process...

1. User input is received.
2. User input is queried against the vector database to retrieve relevant documents.
3. Relevant documents and chat memory are inserted into a new prompt to respond to the user input.
4. Return to step 1.

However, more complex methods of interacting with the user input can generate more accurate results in RAG architectures. One of the popular mechanisms which can increase accuracy of these retrieval systems is utilizing more than one call to an LLM in order to reformat the user input for more effective search to your vector database. A better workflow is described below compared to the one we already built...

1. User input is received.
2. An LLM is used to reword the user input to be a better search query for the vector database based on the chat history and other instructions. This could include things like condensing, rewording, addition of chat context, or stylistic changes.
3. Reformatted user input is queried against the vector database to retrieve relevant documents.
4. The reformatted user input and relevant documents are inserted into a new prompt in order to answer the user question.
5. Return to step 1.

Let's now build out this second workflow using LangChain below.

First we need to make a prompt which will reformat the user input to be more compatible for searching of the vector database. The way we do this is by providing the chat history as well as the some basic instructions to Claude and asking it to condense the input into a single output.

In [ ]:
condense_prompt = PromptTemplate.from_template("""\
<chat-history>
{chat_history}
</chat-history>

<follow-up-message>
{question}
<follow-up-message>

Human: Given the conversation above (between Human and Assistant) and the follow up message from Human, \
rewrite the follow up message to be a standalone question that captures all relevant context \
from the conversation. Answer only with the new question and nothing else.

Assistant: Standalone Question:""")

The next prompt we need is the prompt which will answer the user's question based on the retrieved information. In this case, we provide specific instructions about how to answer the question as well as provide the context retrieved from the vector database.

In [ ]:
respond_prompt = PromptTemplate.from_template("""\
<context>
{context}
</context>

Human: Given the context above, answer the question inside the <q></q> XML tags.

<q>{question}</q>

If the answer is not in the context say "Sorry, I don't know as the answer was not found in the context". Do not use any XML tags in the answer.

Assistant:""")

Now that we have our prompts set up, let's set up the conversational memory buffer just like we did earlier in the notebook. Notice how we inject an example human and assistant message in order to help guide our AI assistant on what its job is.

In [ ]:
llm = Bedrock(
    client=boto3_bedrock,
    model_id="anthropic.claude-instant-v1",
    model_kwargs={"max_tokens_to_sample": 500, "temperature": 0.9}
)
memory_chain = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    human_prefix="Human",
    ai_prefix="Assistant"
)
memory_chain.chat_memory.add_user_message(
    'Hello, what are you able to do?'
)
memory_chain.chat_memory.add_ai_message(
    'Hi! I am a help chat assistant which can answer questions about Amazon SageMaker.'
)

Lastly, we will used the `ConversationalRetrievalChain` from LangChain to orchestrate this whole system. If you would like to see some more logs about what is happening in the orchestration and not just the final output, make sure to change the `verbose` argument to `True`.

In [ ]:
from langchain.chains import ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(
    llm=llm, # this is our claude model
    retriever=vs.as_retriever(), # this is our FAISS vector database
    memory=memory_chain, # this is the conversational memory storage class
    condense_question_prompt=condense_prompt, # this is the prompt for condensing user inputs
    verbose=False, # change this to True in order to see the logs working in the background
)
qa.combine_docs_chain.llm_chain.prompt = respond_prompt # this is the prompt in order to respond to condensed questions

Let's go ahead and generate some responses from our RAG solution!

In [ ]:
display(Markdown(f"{qa.run({'question': 'How can I check for imbalances in my model?'})}"))

In [ ]:
display(Markdown(f"{qa.run({'question': 'What kind does it detect?' })}"))

In [ ]:
display(Markdown(f"{qa.run({'question': 'How does this improve model explainability?' })}"))

## Let us use LLM to validate if the response was factual

#### We first create a sanity prompt, which will use the vector DB results and ask the LLM to validate if the respinse given was acurate or not

In [ ]:
# create sanity check prompt
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate

def create_sanity_prompt(instruction_start: str = None,instruction_end: str = None,) -> PromptTemplate:
    """
    Create a prompt template for LLM sanity check

    Parameters
    ----------
    instruction_start : str, optional
        Instrcution in the beginning of the prompt, by default None
    instruction_end : str, optional
        Instrcution in the end of the prompt, by default None

    Returns
    -------
    PromptTemplate
        Prompt template in the LangChain format
    """

    # first instruction
    prompt_template_build = instruction_start + "\n"

    # add context
    prompt_template_build += "Context: {context}" + "\n"

    # add statement
    prompt_template_build += "Statement: {statement}" + "\n"

    # addinstruction
    prompt_template_build += "Question: " + instruction_end + "\n"

    # add answer placeholder
    prompt_template_build += "Answer:"

    print(prompt_template_build)
    # build the template
    llm_prompt = PromptTemplate(
        template=str(prompt_template_build),
        input_variables=["context", "statement"],
    )
    return llm_prompt

sanity_prompt = create_sanity_prompt(
    instruction_start="""The following is a conversation between a highly knowledgeable and intelligent AI assistant, called Falcon, and a human user asking Questions. In the following interactions, Falcon will converse in natural language, and Falcon will answer the questions based only on the provided Context. Falcon will provide accurate, short and direct answers to the questions.""",
    instruction_end="Is the above statement based directly on the provided context? Answer with yes or no.",
)

docs = vs.similarity_search_with_score('How can I check for imbalances in my model?')
contexts = []
source = []
for doc, score in docs:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")
    if score <= 0.9:
        contexts.append(doc)
        source.append(doc.metadata['source'])
        print(f"\n INPUT CONTEXT:{contexts}")

sanity_chain = load_qa_chain(llm=llm, prompt=sanity_prompt)
sanity_check = sanity_chain({"input_documents": contexts, "statement": output},return_only_outputs=True)['output_text']

sanity_check

#### We can see the Vector database responded accurately to our query

In [ ]:
sanity_check

--- 
## Using LlamaIndex for Orchestration of RAG

Another popular open source framework for orchestrating RAG is [LlamaIndex](https://gpt-index.readthedocs.io/en/latest/index.html). Let's take a look below at how to use our SageMaker FAQ vector index to have a conversational RAG application with LlamaIndex.

In [ ]:
from IPython.display import Markdown, display
from langchain.embeddings.bedrock import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

from llama_index import ServiceContext

First we need to set up the system setting to define the embedding model and LLM. Again, we will be using titan and claude respectively.

In [ ]:
embed_model = BedrockEmbeddings(client=boto3_bedrock, model_id="amazon.titan-embed-text-v1")
llm = Bedrock(
    client=boto3_bedrock,
    model_id="anthropic.claude-instant-v1",
    model_kwargs={
        "max_tokens_to_sample": 500,
        "temperature": 0.9,
    },
)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model, chunk_size=512
)

The next step would be to create a FAISS index from our document base. In this lab, this is already done for you and stored in the [faiss-index/llama-index/](../faiss-index/llama-index/) folder.

If you are interested in how this was accomplished, follow [this tutorial](https://gpt-index.readthedocs.io/en/latest/examples/vector_stores/FaissIndexDemo.html) from LlamIndex. The code below is the basics of how this was accomplished as well.

```python
# faiss_index = faiss.IndexFlatL2(1536)
# vector_store = FaissVectorStore(faiss_index=faiss_index)
# documents = SimpleDirectoryReader("./../data/sagemaker").load_data()
# vector_store = FaissVectorStore(faiss_index=faiss_index)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context, service_context=service_context
# )
# index.storage_context.persist('../faiss-index/llama-index/')
```

Once the index is created, we can load the persistent files to a `FaissVectorStore` object and create a `query_engine` from the vector index. To learn more about indicies in LlamaIndex, read more [here](https://gpt-index.readthedocs.io/en/latest/understanding/indexing/indexing.html).

In [ ]:
from llama_index import load_index_from_storage, StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore

vector_store = FaissVectorStore.from_persist_dir("./faiss-index/llama-index")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./faiss-index/llama-index"
)
index = load_index_from_storage(storage_context=storage_context, service_context=service_context)
query_engine = index.as_query_engine()

Now let's set up a retrieval based chat application similar to LangChain. We will use the same condensing question strategy as before and can reuse the same prompt to condense the question for vector searching. Notice how we include some custom chat history to inject context into the prompt for the model to understand what we are asking questions about. The resulting `chat_engine` object is now fully ready to chat about our documents.

In [ ]:
from llama_index.prompts  import PromptTemplate
from llama_index.llms import ChatMessage, MessageRole
from llama_index.chat_engine.condense_question import CondenseQuestionChatEngine

custom_prompt = PromptTemplate("""\
<chat-history>
{chat_history}
</chat-history>

<follow-up-message>
{question}
<follow-up-message>

Human: Given the conversation above (between Human and Assistant) and the follow up message from Human, \
rewrite the message to be a standalone question that captures all relevant context \
from the conversation. Answer only with the new question and nothing else.

Assistant: Standalone Question:""")

custom_chat_history = [
    ChatMessage(
        role=MessageRole.USER,
        content='Hello assistant, I have some questions about using Amazon SageMaker today.'
    ),
    ChatMessage(
        role=MessageRole.ASSISTANT,
        content='Okay, sounds good.'
    )
]

query_engine = index.as_query_engine()
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=custom_prompt,
    chat_history=custom_chat_history,
    service_context=service_context,
    verbose=True
)

Let's go ahead and ask our first question. Notice that the verbose `chat_engine` will print out the condensed question as well.

In [ ]:
response = chat_engine.chat("How can I check for imbalances in my model?")

In [ ]:
display(Markdown(f"{response}"))

Now follow up questions can be asked with conversational context in mind!

In [ ]:
response = chat_engine.chat("How does this improve model explainability?")

In [ ]:
display(Markdown(f"{response}"))

---
## Next steps

Now that we have a working RAG application with vector search retrieval, we will explore a new type of retrieval. In the next notebook we will see how to use LLM agents to automatically retrieve information from APIs.